# LCA model for electric drone

## Project setup

In [ ]:
# Import libraries
import brightway2 as bw
import os 
import lca_algebraic as agb
from sympy import init_printing
import matplotlib.pyplot as plt

# Pretty print for Sympy
init_printing()

# Set current project
bw.projects.set_current('LCA_course')

# Init user database
USER_DB = 'Foreground Drone'
agb.resetDb(USER_DB)

## LCA model creation

In [ ]:
### Définition des paramètres

# Energie consommée sur une mission
mission_energy = agb.newFloatParam(  # ce paramètre est un nombre (float)
    'mission_energy',  # nom du paramètre
    default=0.180,  # valeur par défaut
    unit='kWh',  # unité
    min=0, max=1000,  # bornes sur ce paramètre (non utilisé ici)
    description="energy consumption for a single mission",  # description
    dbname=USER_DB  # on enregistre le paramètre dans notre base de données foreground
)

# Durée de vie du drone, en nombre de missions réalisées --> pour calculer l'énergie totale consommée sur la durée de vie du drone
n_missions = agb.newFloatParam(
    'n_missions',
    default=2500.0,
    min=0, max=10000,
    description="number of missions over drone lifetime",
    dbname=USER_DB
)

# Mix électrique
elec_switch_param = agb.newEnumParam(  # ce paramètre est un switch
    'elec_mix',
    values=['us', 'eu', 'fr'], # valeurs possibles de ce paramètre
    default='eu',
    description="switch on electricty mix",
    dbname=USER_DB
)

# Masse des hélices
mass_propellers = agb.newFloatParam(
    'mass_propellers',
    default=0.43,
    unit='kg',
    min=0, max=1000,
    description="mass of propellers",
    dbname=USER_DB
)

# Masse de moteurs
mass_motors = agb.newFloatParam(
    'mass_motors',
    default=0.52,
    unit='kg',
    min=0, max=1000,
    description="mass of motors",
    dbname=USER_DB
)

# Masse de la structure
mass_structure = agb.newFloatParam(
    'mass_structure',
    default=0.57,
    unit='kg',
    min=0, max=1000,
    description="mass of structure",
    dbname=USER_DB
)

# Masse de la batterie
mass_batteries = agb.newFloatParam(
    'mass_batteries',
    default=1.08,
    unit='kg',
    min=0, max=1000,
    description="mass of batteries",
    dbname=USER_DB
)

# Durée de vie de la batterie, en nombre de cycles
n_cycles_battery = agb.newFloatParam(  # ce paramètre est un nombre (float)
    'n_cycles_battery',  # nom du paramètre
    default=500.0,  # valeur par défaut
    min=1, max=10000,  # bornes sur ce paramètre (non utilisé ici)
    description="number of cycles for the battery",
    dbname=USER_DB
)

# Types de batterie
battery_type = agb.newEnumParam(
    'battery_type',
    values=["nmc", "lfp"],  # valeurs possibles de ce paramètre (différentes technologies de batteries)
    default="nmc",
    description="battery technology",
    dbname=USER_DB
)

agb.list_parameters()

In [ ]:
# Background activities
battery_nmc = agb.findTechAct("Market for battery cell, Li-ion, NMC811")
battery_lfp = agb.findTechAct("Market for battery cell, Li-ion, LFP")
electric_motor = agb.findTechAct("Market for electric motor, for electric scooter")
composite = agb.findTechAct("Market for carbon fibre reinforced plastic, injection moulded")
electricity_eu = agb.findTechAct("Market group for electricity, low voltage", loc='Europe without Switzerland')
electricity_us = agb.findTechAct("Market group for electricity, low voltage", loc='US')
electricity_fr = agb.findTechAct("Market for electricity, low voltage", loc='FR')

In [ ]:
from sympy import ceiling

### NIVEAU -2 ###
# Création du processus "battery" pour sélectionner le type de batterie et le nombre de batteries nécessaires au cours de la vie du drone
battery = agb.newSwitchAct(
    USER_DB,    # on enregistre le processus dans notre base de données foreground 
    "battery",  # nom du processus
    battery_type,  # paramètre de switch précedemment défini
    { # processus à sélectionner selon de la valeur du switch
        "nmc": battery_nmc,
        "lfp": battery_lfp,
    }
)
battery.updateMeta(subphase="batteries")

# Création du processus "propeller" pour définir le type d'hélice du drone
propeller = agb.newActivity(
    USER_DB,
    "propeller",
    "kg",  # unité
    exchanges={  # échanges avec les autres processus, sous la forme (processus : quantité fixe ou définie à partir d'une formule)
        composite: 1.0,  # pour 1 kg d'hélices on a besoin de 1 kg de composite
        # composite: 0.8  # on peut également définir une valeur inférieure à 1.0 pour prendre en compte le recyclage
    }
)
propeller.updateMeta(subphase="propellers")

# Création du processus "motor"
motor = agb.newActivity(
    USER_DB, 
    "motor",
    "kg",
    exchanges={
        electric_motor: 1.0,
    }
)
motor.updateMeta(subphase="motors")

# Création du processus "structures"
structure = agb.newActivity(
    USER_DB,
    "structure",
    "kg",
    exchanges={
        composite: 1.0,
        # aluminium: 0.8,
    }
)
structure.updateMeta(subphase="structures")

### NIVEAU -1 ###
# Création du processus "production d'un drone"
production = agb.newActivity(
    USER_DB, 
    "production", 
    "drone unit", # Unité
    exchanges= { # échanges avec les autres processus, sous la forme (processus : quantité définie à partir d'une formule)
        structure: mass_structure, 
        propeller: mass_propellers,
        motor: mass_motors,
        battery: mass_batteries * ceiling(n_missions / n_cycles_battery),  # on référence le processus "battery" que l'on vient de créer
    })
production.updateMeta(phase="drone production")

# Création du processus "operation" pour la consommation d'énergie en vol.
operation = agb.newSwitchAct(
    USER_DB, 
    "operation",
    elec_switch_param, # paramètre de switch
    {
        "us" : electricity_us,
        "eu" : electricity_eu,
        "fr" : electricity_fr,
    })
operation.updateMeta(phase="use phase")
operation.updateMeta(subphase="electricity")


### NIVEAU 0 ###
# Dernière pièce du puzzle: création du processus principal permettant de fournir une unité fonctionnelle
model = agb.newActivity(
    USER_DB,
    "package transport, drone",
    "one package delivery",  # impacts exprimés par colis livré
    exchanges= {
        production: 1.0 / n_missions,  # on a besoin de 1 drone...
        operation: mission_energy,  # ...et d'une certaine quantité d'électricité
    }
)

In [ ]:
# Plot process tree
from helpers import *

NETWORK_PATH = './lca_activities.html'
graph_activities(USER_DB, model, NETWORK_PATH)

from IPython.display import IFrame
IFrame(src='lca_activities.html', width="100%", height="500px")

In [ ]:
# Impact calculation test

# List of impact methods to consider
impact_method = agb.findMethods("climate change", mainCat="EF v3.1")[0]

# Compute impacts
agb.compute_impacts(
    
    # Activity/Process to assess
    model, 
    
    # list of impacts to consider
    [impact_method], 

    axis="subphase"
)

## Export model

In [ ]:
# Save database and parameters as Bzipped JSON
agb.export_db(USER_DB, "./db_drone.bw2")